# Compare with real ground truth - Pulldown

In [1]:
seed = 123
import sys
from pathlib import Path
from typing import *
from Bio import SeqIO
import pandas as pd
import torch
import numpy as np
import os
from tqdm.notebook import tqdm
import argparse
import matplotlib.pyplot as plt
from torch.utils.data import random_split, Subset, DataLoader, DistributedSampler

sys.path.insert(0, "..")
from dataset.data import (
    RNADatasetInference,
    seed_everything,
)
import util.misc as utils
from models.binary_classifier import build as build_model
from train_binary_cl import (
    get_args_parser,
)

<class 'transformers.tokenization_dna.DNATokenizer'>


In [2]:
ROOT_DIR = os.path.dirname(os.path.abspath('.'))
original_files_dir = os.path.join(ROOT_DIR, "dataset", "original_files")
processed_files_dir = os.path.join(ROOT_DIR, "dataset", "processed_files")
external_dataset_files_dir = os.path.join(ROOT_DIR, "dataset", "external_dataset", "pulldown")

In [3]:
sys.argv = ['']
parser = argparse.ArgumentParser('Training', parents=[get_args_parser()])
args = parser.parse_args()

In [4]:
step_size = 350
device = 'cuda:0'
num_workers = 10
batch_size = 32
args.output_dir = os.path.join(ROOT_DIR, 'checkpoints', 'binary_cl')
args.dataset_path = os.path.join(ROOT_DIR, 'dataset')
args.resume = os.path.join(args.output_dir, 'checkpoint.pth')
args.resume = args.resume.replace('checkpoint.pth', 'best_model.pth')
bert_pretrained_dir = os.path.join(ROOT_DIR, 'dataset', 'pre_trained_DNABERT', '6-new-12w-0')
ufold_dir = os.path.join(ROOT_DIR, 'UFold_dependencies')
ufold_path= os.path.join(ufold_dir, 'models', 'ufold_train_alldata.pt')
assert os.path.isfile(args.resume)

In [5]:
seed_everything(seed)
dataset = RNADatasetInference(
    gene_info_path = os.path.join(external_dataset_files_dir, "genes.fa"),
    interactions_path = os.path.join(external_dataset_files_dir, "pairs.txt"),
    dot_bracket_path = os.path.join(external_dataset_files_dir, "dot_bracket.txt"),
    step_size=step_size
)

sampler = torch.utils.data.SequentialSampler(dataset)

data_loader = DataLoader(dataset, batch_size = args.batch_size, sampler=sampler,
                             drop_last=False, collate_fn=utils.collate_fn, num_workers=args.num_workers)

In [6]:
model, criterion, postprocessors = build_model(args, bert_pretrained_dir, ufold_path)
model.to(device)
model_without_ddp = model

checkpoint = torch.load(args.resume, map_location='cpu')
model_without_ddp.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [8]:
probability = []
g1 = []
g2 = []
len_g1 = []
len_g2 = []
x1 = []
x2 = []
y1 = []
y2 = []

with torch.no_grad():
    for samples, targets in tqdm(data_loader):
        rna1, rna2 = samples
        rna1[0] = rna1[0].to(device)
        rna2[0] = rna2[0].to(device)
        rna1[1].tensors = rna1[1].tensors.to(device)
        rna2[1].tensors = rna2[1].tensors.to(device)
        rna1[1].mask = rna1[1].mask.to(device)
        rna2[1].mask = rna2[1].mask.to(device)
        
        outputs = model(rna1, rna2)
        probability += outputs['cnn_output'].softmax(-1)[:, 1].tolist()
        g1 += [t['gene1'] for t in targets]
        g2 += [t['gene2'] for t in targets]
        len_1 = np.array([(t['bbox'].x2 - t['bbox'].x1) for t in targets])
        len_2 = np.array([(t['bbox'].y2 - t['bbox'].y1) for t in targets])
        len_g1 += list(len_1)
        len_g2 += list(len_2)
        x1 +=[t['bbox'].x1 for t in targets]
        x2 +=[t['bbox'].x2 for t in targets]
        y1 +=[t['bbox'].y1 for t in targets]
        y2 +=[t['bbox'].y2 for t in targets]

res = pd.DataFrame({
    'probability':probability,
    'g1':g1,
    'g2':g2,
    'x1':x1,
    'x2':x2,
    'y1':y1,
    'y2':y2,
    'len_g1': len_g1,
    'len_g2': len_g2,
})

  0%|          | 0/156 [00:00<?, ?it/s]

In [12]:
pulldown_dir = os.path.join(ROOT_DIR, 'dataset', 'external_dataset', 'pulldown')
pos = os.path.join(pulldown_dir, 'cand_99.ids.fa')
neg = os.path.join(pulldown_dir, 'ctrl_99.ids.fa')

fasta_sequences = [SeqIO.parse(open(pos),'fasta'), 
                   SeqIO.parse(open(neg), 'fasta'),
                  ]

pos_samples = []
neg_samples = []
for k in range(len(fasta_sequences)):
    for _, fasta in enumerate(fasta_sequences[k]):
        name = str(fasta.description)
        if k == 0:
            pos_samples.append(name)
        elif k == 1:
            neg_samples.append(name)

In [13]:
neg = res[res.g2.isin(neg_samples)]
print(neg.shape[0])

4140


In [14]:
print(neg[neg.probability < 0.5].shape[0]/neg.shape[0])
print(neg.probability.mean())

d = {}
for g2 in neg.g2.unique():
    subset = neg[neg.g2 == g2]
    d[g2] = {'n_tot':subset.shape[0],
            'predicted_pos':subset[subset.probability>0.5].shape[0]}
df_neg = pd.DataFrame.from_dict(d, orient = 'index').reset_index().rename({'index':'gene'}, axis = 1)

0.5219806763285024
0.4587731621181221


In [16]:
pos = res[res.g2.isin(pos_samples)]
print(pos.shape[0])

826


In [17]:
print(pos[pos.probability > 0.5].shape[0]/pos.shape[0])
print(pos.probability.mean())
d = {}
for g2 in pos.g2.unique():
    subset = pos[pos.g2 == g2]
    d[g2] = {'n_tot':subset.shape[0],
            'predicted_pos':subset[subset.probability>0.5].shape[0]}
df_pos = pd.DataFrame.from_dict(d, orient = 'index').reset_index().rename({'index':'gene'}, axis = 1)

0.47578692493946734
0.45637122840173205


In [18]:
df_neg.predicted_pos.mean()

7.067857142857143

In [19]:
df_neg[df_neg['predicted_pos']>0].shape[0]/df_neg.shape[0]

0.925

In [20]:
df_pos.predicted_pos.mean()

7.1454545454545455

In [21]:
df_pos[df_pos['predicted_pos']>0].shape[0]/df_pos.shape[0]

0.9272727272727272